## calculate change ratios and combine common data

### feature(주요 common data) 항목 변화율 계산하고 합치기-- 하루전과 이틀전의 데이터와 비교

In [1]:
import datetime, time
from datetime import date

import pandas as pd
import numpy as np

In [2]:
# 개장일(date)과 이전 개장일(date_p1), 이전이전 개장일(date_p2)을 dict로 구성
base_data_directory = './data/base_data/stock_market_holydays/'
opening_days_kor = pd.read_pickle(base_data_directory+'opening_days_kor.pkl') # 한국 개장일 데이터 
opening_days_usa = pd.read_pickle(base_data_directory+'opening_days_usa.pkl') # 미국 개장일 데이터 

In [3]:
# 매일 변화가 있는 항목 추출; 금리변동과 같이 주기적을 발생하는 항목들(cpi, bok_rate, fed_rate 등)은 제외
common_pkl = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
               "ixic_future.pkl", "kor_10yr_bond.pkl",
              "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", "nas.pkl",
              "snp_future.pkl", "sox.pkl", "spx.pkl", "us_10yr_bond.pkl", "us_2yr_bond.pkl",
              "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
              'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
              'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
              'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
common_pkl_usa = [ "dji.pkl", "dji_future.pkl", "dxy_future.pkl", 
                  "ixic_future.pkl", "nas.pkl", "snp_future.pkl", "sox.pkl", "spx.pkl", 
                  "us_10yr_bond.pkl", "us_2yr_bond.pkl", "us_3mon_bond.pkl", "vix.pkl", "wti_future.pkl",
                  'spsy.pkl', 'spny.pkl', 'spxhc.pkl', 'splrcd.pkl', 'splrci.pkl', 'splrcu.pkl', 'splrcs.pkl', 
                  'splrct.pkl', 'splrcl.pkl', 'splrcm.pkl', 'ixbk.pkl', 'ixfn.pkl', 'ixid.pkl', 'ixis.pkl', 
                  'ixk.pkl', 'ixtr.pkl', 'ixut.pkl', 'nbi.pkl', 'bkx.pkl' 
             ]
common_pkl_kor = [ "kor_10yr_bond.pkl", "kor_2yr_bond.pkl", "kosdaq.pkl", "kospi.pkl", "krw_rate.pkl", 
             ]
# common_pkl = [ "dji.pkl" ]

In [4]:
# dict로 만들어 pkl내에 있는 df의 column 이름과 매칭시키기 위하여 구성
common_col_name = {'dji.pkl':'dji', 'dji_future.pkl':'dji_f', 'dxy_future.pkl':'dxy', 
                   'ixic_future.pkl':'ixic_f', 'kor_10yr_bond.pkl':'bond_kor_10',
                   'kor_2yr_bond.pkl':'bond_kor_2', 'kosdaq.pkl':'kosdaq', 'kospi.pkl':'kospi',
                   'krw_rate.pkl':'krw', 'nas.pkl':'ixic', 'snp_future.pkl':'spx_f',
                   'sox.pkl':'sox', 'spx.pkl':'spx', 'us_10yr_bond.pkl':'bond_usa_10',
                   'us_2yr_bond.pkl':'bond_usa_2', 'us_3mon_bond.pkl':'bond_usa_3m',
                   'vix.pkl':'vix', 'wti_future.pkl':'wti',
                   'spsy.pkl':'spsy', 'spny.pkl':'spny', 'spxhc.pkl':'spxhc', 'splrcd.pkl':'splrcd', 
                   'splrci.pkl':'splrci', 'splrcu.pkl':'splrcu', 'splrcs.pkl':'splrcs', 
                   'splrct.pkl':'splrct', 'splrcl.pkl':'splrcl', 'splrcm.pkl':'splrcm', 
                   'ixbk.pkl':'ixbk', 'ixfn.pkl':'ixfn', 'ixid.pkl':'ixid', 'ixis.pkl':'ixis', 
                   'ixk.pkl':'ixk', 'ixtr.pkl':'ixtr', 'ixut.pkl':'ixut', 'nbi.pkl':'nbi', 'bkx.pkl':'bkx'
                   }

In [5]:
directory_for_predict = './data/data_for_ml/predict/'
pkl_common_directory = './data/common_pkl/'
pkl_directory = './data/common_pkl/'

In [6]:
# 주요 지표를 한개의 df로 구성 (휴장일과 개장일이 틀리기 때문에 한국, 미국 각각 구성해야 함)
# _common : 공통으로 사용되는 지표를 의미
def get_common_base(opening_days_common, pkl_list):
    df_base = pd.DataFrame(pd.Series(opening_days_common, name='date')).set_index('date')
    date_compare = pd.DataFrame(pd.Series(opening_days_common, name='date'))
    date_compare['date_p'] = date_compare['date'].shift(1)
    
    date_compare.columns = ['date_c', 'date_p'] #거래일과 바로 이전 거래일
    c_p_dict = date_compare.set_index('date_c').to_dict()['date_p'] # date_c로 date_p 찾기
    p_c_dict = date_compare.set_index('date_p').to_dict()['date_c'] # date_p로 date_c 찾기
    
    for index_name in pkl_list:
        key_name = index_name[:-4]
        col_name = common_col_name[index_name]
        df_temp = pd.read_pickle(pkl_common_directory + index_name)
        
#         df_temp['date'] = df_temp['date'].dt.date # date type으로 변환
        df_temp['temp'] = df_temp[col_name].shift(2)  # 거래일과 이전(-1) 이전(-1) 거래일 데이터와 비교하기 위하여 -2 shift
        
        df_temp[f'{col_name}_cr_2'] = (df_temp[col_name] - df_temp['temp'])/df_temp['temp']*100 # 이전 거래일 데이터는 "_cr"
        
        df_new = df_temp[df_temp['date'].isin(date_compare['date_p'])] # 이전 거래일과 비교날짜가 있는 row만 선택
        # 예: 2023-04-10('date_c'), 2023-04-07('date_p') 일 경우, 2023년 4월 7일 데이터는 2023년 4월 10일 row에 작성해야 함.
        # 위 예와 같이 하기 위해 'date_p'에 있는 date로만 df재구성(df_new)

        add_c= df_new['date'].apply(lambda x : p_c_dict[x])  # 하루전 날짜와 매칭한 column 생성 (add_c)
        # 예: 2023년 4월 7일 데이터는 2023년 4월 10일 row에 기입
        df_new.insert(len(df_new.columns)-1, 'date_c', value=add_c)  #SettingWithCopyWarning: error 회피
        df_new.set_index('date_c', inplace=True)

        df_base = df_base.merge(df_new.iloc[:, [-3, -1]], \
                                how='left', left_index=True, right_index=True)
    return df_base

In [7]:
# ***미국 주식 분석용 시작 일자, 마지막 일자  지정 ***********
# ** 한국 공통(kospi 등) 자료는 한국 개장일에 맞춰 분석해야 함.
pkl_name= 'dji.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

dji = df_temp['dji'].iloc[-1]
opening_days_common = list(df_temp['date'])

# 오늘 날짜(분석날짜) 추가 필요(미국, 한국 포함)

if dji == None :
    opening_days_common = list(df_temp['date'][:-1])
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(opening_days_usa).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(opening_days_usa)[lp1_index])

In [8]:
# 미국 common data df
df_usa = get_common_base(opening_days_common, common_pkl_usa)

+++ dji.pkl
+++11 dji.pkl
+++111 dji.pkl
+++1111 dji.pkl
+++22 dji.pkl
+++33 dji.pkl
+++44 dji.pkl
+++ dji_future.pkl
+++11 dji_future.pkl
+++111 dji_future.pkl
+++1111 dji_future.pkl
+++22 dji_future.pkl
+++33 dji_future.pkl
+++44 dji_future.pkl
+++ dxy_future.pkl
+++11 dxy_future.pkl
+++111 dxy_future.pkl
+++1111 dxy_future.pkl
+++22 dxy_future.pkl
+++33 dxy_future.pkl
+++44 dxy_future.pkl
+++ ixic_future.pkl
+++11 ixic_future.pkl
+++111 ixic_future.pkl
+++1111 ixic_future.pkl
+++22 ixic_future.pkl
+++33 ixic_future.pkl
+++44 ixic_future.pkl
+++ nas.pkl
+++11 nas.pkl
+++111 nas.pkl
+++1111 nas.pkl
+++22 nas.pkl
+++33 nas.pkl
+++44 nas.pkl
+++ snp_future.pkl
+++11 snp_future.pkl
+++111 snp_future.pkl
+++1111 snp_future.pkl
+++22 snp_future.pkl
+++33 snp_future.pkl
+++44 snp_future.pkl
+++ sox.pkl
+++11 sox.pkl
+++111 sox.pkl
+++1111 sox.pkl
+++22 sox.pkl
+++33 sox.pkl
+++44 sox.pkl
+++ spx.pkl
+++11 spx.pkl
+++111 spx.pkl
+++1111 spx.pkl
+++22 spx.pkl
+++33 spx.pkl
+++44 spx.pkl
+++ u

In [9]:
# *****한국 주식 시작 일자, 마지막 일자  지정 ***********
pkl_name= 'kospi.pkl'
df_temp = pd.read_pickle(pkl_directory + pkl_name)

kospi = df_temp['kospi'].iloc[-1]
opening_days_common = list(df_temp['date'])

if kospi == None :
    opening_days_common = list(df_temp['date'][:-1])
    
# 마지막 기록일자 index 찾고 다음 개장일 추가하기 (예측을 원하는 날)
l_date = opening_days_common[-1]
lp1_index = list(opening_days_kor).index(l_date) + 1 # 현재 개장일 이후에 오는 개장일 날짜 index (+1 index)
opening_days_common.append(list(opening_days_kor)[lp1_index])

In [10]:
# 한국 common data df
df_kor = get_common_base(opening_days_common, common_pkl_kor)

+++ kor_10yr_bond.pkl
+++11 kor_10yr_bond.pkl
+++111 kor_10yr_bond.pkl
+++1111 kor_10yr_bond.pkl
+++22 kor_10yr_bond.pkl
+++33 kor_10yr_bond.pkl
+++44 kor_10yr_bond.pkl
+++ kor_2yr_bond.pkl
+++11 kor_2yr_bond.pkl
+++111 kor_2yr_bond.pkl
+++1111 kor_2yr_bond.pkl
+++22 kor_2yr_bond.pkl
+++33 kor_2yr_bond.pkl
+++44 kor_2yr_bond.pkl
+++ kosdaq.pkl
+++11 kosdaq.pkl
+++111 kosdaq.pkl
+++1111 kosdaq.pkl
+++22 kosdaq.pkl
+++33 kosdaq.pkl
+++44 kosdaq.pkl
+++ kospi.pkl
+++11 kospi.pkl
+++111 kospi.pkl
+++1111 kospi.pkl
+++22 kospi.pkl
+++33 kospi.pkl
+++44 kospi.pkl
+++ krw_rate.pkl
+++11 krw_rate.pkl
+++111 krw_rate.pkl
+++1111 krw_rate.pkl
+++22 krw_rate.pkl
+++33 krw_rate.pkl
+++44 krw_rate.pkl


In [11]:
df_all = df_usa.merge(df_kor, how='left', left_index=True, right_index=True)

In [12]:
# write common analysis data
globals()['df_all'].to_pickle(directory_for_predict + '0_df_common.pkl')
globals()['df_all'].to_csv(directory_for_predict + '0_df_common.csv')

In [13]:
df_all

,dji_cr,dji_cr_2,dji_f_cr,dji_f_cr_2,dxy_cr,dxy_cr_2,ixic_f_cr,ixic_f_cr_2,ixic_cr,ixic_cr_2,...,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2
date,,,,,,,,,,,,,,,,,,,,,
2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-0.23%,2.886406,-1.87%,0.781250,NaN,NaN,0.48,0.936512,+0.04%,-0.354559
2021-12-28,nan%,NaN,nan%,NaN,nan%,NaN,nan%,NaN,nan%,NaN,...,0.45%,0.454133,0.18%,0.179856,NaN,NaN,-0.43,0.046028,-0.03%,0.005061
2021-12-29,0.26%,NaN,0.29%,NaN,0.12%,NaN,-0.43%,NaN,-0.56%,NaN,...,-0.72%,-0.272480,0.66%,0.839329,NaN,NaN,0.69,0.259259,+0.21%,0.177067
2021-12-30,0.25%,0.513052,0.27%,0.561114,-0.29%,-0.166513,0.02%,-0.419686,-0.10%,-0.661825,...,-0.73%,-1.446655,-0.36%,0.299222,NaN,NaN,-0.89,-0.208698,-0.29%,-0.076754
2021-12-31,-0.25%,-0.000365,-0.25%,0.024806,0.04%,-0.248439,-0.37%,-0.350255,-0.16%,-0.254473,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-25,-0.77%,-1.462019,-0.84%,-1.516787,0.40%,0.698495,-0.57%,-1.814422,-0.61%,-1.860107,...,+0.92%,2.971462,+0.06%,1.084090,-0.43,0.40139,-0.0,0.405541,-0.06%,0.356232
2023-05-26,-0.11%,-0.879918,-0.15%,-0.983943,0.36%,0.764135,2.39%,1.804607,1.71%,1.097429,...,+2.80%,3.748558,+3.01%,3.074246,-0.9,-1.331533,-0.5,-0.500867,+0.97%,0.908119
2023-05-30,1.00%,0.894569,0.97%,0.824862,-0.04%,0.324664,2.55%,4.998077,2.19%,3.937231,...,+1.03%,3.857143,+1.63%,4.695652,-0.53,-1.42964,0.16,-0.336521,-0.64%,0.320836


In [14]:
l_date

datetime.date(2023, 5, 26)

In [15]:
list(opening_days_kor).index(l_date)

839

In [16]:
opening_days_common

[datetime.date(2021, 11, 29),
 datetime.date(2021, 11, 30),
 datetime.date(2021, 12, 1),
 datetime.date(2021, 12, 2),
 datetime.date(2021, 12, 3),
 datetime.date(2021, 12, 6),
 datetime.date(2021, 12, 7),
 datetime.date(2021, 12, 8),
 datetime.date(2021, 12, 9),
 datetime.date(2021, 12, 10),
 datetime.date(2021, 12, 13),
 datetime.date(2021, 12, 14),
 datetime.date(2021, 12, 15),
 datetime.date(2021, 12, 16),
 datetime.date(2021, 12, 17),
 datetime.date(2021, 12, 20),
 datetime.date(2021, 12, 21),
 datetime.date(2021, 12, 22),
 datetime.date(2021, 12, 23),
 datetime.date(2021, 12, 24),
 datetime.date(2021, 12, 27),
 datetime.date(2021, 12, 28),
 datetime.date(2021, 12, 29),
 datetime.date(2021, 12, 30),
 datetime.date(2022, 1, 3),
 datetime.date(2022, 1, 4),
 datetime.date(2022, 1, 5),
 datetime.date(2022, 1, 6),
 datetime.date(2022, 1, 7),
 datetime.date(2022, 1, 10),
 datetime.date(2022, 1, 11),
 datetime.date(2022, 1, 12),
 datetime.date(2022, 1, 13),
 datetime.date(2022, 1, 14),
 d

In [29]:
df_kor

,bond_kor_10_cr,bond_kor_10_cr_2,bond_kor_2_cr,bond_kor_2_cr_2,kosdaq_cr,kosdaq_cr_2,kospi_cr,kospi_cr_2,krw_cr,krw_cr_2
date,,,,,,,,,,
2021-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-28,0.45%,0.454133,0.18%,0.179856,nan%,NaN,nan%,NaN,-0.03%,0.005061
2021-12-29,-0.72%,-0.272480,0.66%,0.839329,1.59%,NaN,0.69%,NaN,+0.21%,0.177067
2021-12-30,-0.73%,-1.446655,-0.36%,0.299222,0.06%,1.650259,-0.89%,-0.208698,-0.29%,-0.076754
2022-01-04,3.12%,2.367942,-0.24%,-0.594530,0.58%,0.636537,-0.52%,-1.410156,+0.39%,0.106894
...,...,...,...,...,...,...,...,...,...,...
2023-04-28,+0.97%,0.571945,+0.63%,0.782426,2.38%,1.371153,0.44%,0.272800,+0.09%,-0.128273
2023-05-02,+0.42%,1.390148,+0.42%,1.051683,-0.87%,1.491982,0.23%,0.672076,-0.10%,-0.017190
2023-05-03,+0.15%,0.148500,+1.13%,1.128266,1.52%,0.635133,0.91%,1.145113,0.00%,0.301905
